# ADA Homework 03 - Interactive Viz

In [168]:
# Import libraries
import pandas as pd
import numpy as np

# Map
import folium

# Query with geonames
import json
import requests
from urllib import parse 
from urllib.request import urlopen

# Ignore warning
import warnings
warnings.filterwarnings('ignore')

# folium 
import folium

> **Assignemnt**

> Build a Choropleth map which shows intuitively (i.e., use colors wisely) how much grant money goes to each Swiss canton. To do so, you will need to use the provided TopoJSON file, combined with the Choropleth map example you can find in the Folium README file.

## 1. Import data

In [95]:
# The data is downloaded from: http://p3.snf.ch/Pages/DataAndDocumentation.aspx
# The downloaded csv file use semi-colons ';' as delimeter and contains
# the column header has first row
p3_grand = pd.read_csv("Data/P3_GrantExport.csv", sep=";", index_col=0)
p3_grand.head()

,Project Title,Project Title English,Responsible Applicant,Funding Instrument,Funding Instrument Hierarchy,Institution,University,Discipline Number,Discipline Name,Discipline Name Hierarchy,Start Date,End Date,Approved Amount,Keywords
"﻿""Project Number""",,,,,,,,,,,,,,
1,Schlussband (Bd. VI) der Jacob Burckhardt-Biog...,NaN,Kaegi Werner,Project funding (Div. I-III),Project funding,NaN,Nicht zuteilbar - NA,10302,Swiss history,Human and Social Sciences;Theology & religious...,01.10.1975,30.09.1976,11619.00,NaN
4,Batterie de tests à l'usage des enseignants po...,NaN,Massarenti Léonard,Project funding (Div. I-III),Project funding,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,10104,Educational science and Pedagogy,"Human and Social Sciences;Psychology, educatio...",01.10.1975,30.09.1976,41022.00,NaN
5,"Kritische Erstausgabe der ""Evidentiae contra D...",NaN,Kommission für das Corpus philosophorum medii ...,Project funding (Div. I-III),Project funding,Kommission für das Corpus philosophorum medii ...,"NPO (Biblioth., Museen, Verwalt.) - NPO",10101,Philosophy,Human and Social Sciences;Linguistics and lite...,01.03.1976,28.02.1985,79732.00,NaN
6,Katalog der datierten Handschriften in der Sch...,NaN,Burckhardt Max,Project funding (Div. I-III),Project funding,Abt. Handschriften und Alte Drucke Bibliothek ...,Universität Basel - BS,10302,Swiss history,Human and Social Sciences;Theology & religious...,01.10.1975,30.09.1976,52627.00,NaN
7,Wissenschaftliche Mitarbeit am Thesaurus Lingu...,NaN,Schweiz. Thesauruskommission,Project funding (Div. I-III),Project funding,Schweiz. Thesauruskommission,"NPO (Biblioth., Museen, Verwalt.) - NPO",10303,Ancient history and Classical studies,Human and Social Sciences;Theology & religious...,01.01.1976,30.04.1978,120042.00,NaN


In [96]:
p3_grand.index.is_unique #the project number is unique, thus we set it as the index of our table

True

The data contains 15 columns. Among these columns, there are 8 columns containing some 'NaN' values.

In [97]:
p3_grand.isnull().any()

Project Title                   False
Project Title English            True
Responsible Applicant           False
Funding Instrument              False
Funding Instrument Hierarchy     True
Institution                      True
University                       True
Discipline Number               False
Discipline Name                 False
Discipline Name Hierarchy        True
Start Date                       True
End Date                         True
Approved Amount                 False
Keywords                         True
dtype: bool

## 2. Data filter and cleaning

### 2.1 Column Selection

In order to draw the Choropleth map, we need to have the canton locations and the grant amount from the data.<br>
We will use column **Institution** and **University** to get canton locations later, as for the grant amount, we keep the column **Approved Amount**.

In [98]:
data_of_interest = p3_grand[['Institution', 'University', 'Approved Amount']]
data_of_interest.head()

,Institution,University,Approved Amount
"﻿""Project Number""",,,
1,NaN,Nicht zuteilbar - NA,11619.00
4,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,41022.00
5,Kommission für das Corpus philosophorum medii ...,"NPO (Biblioth., Museen, Verwalt.) - NPO",79732.00
6,Abt. Handschriften und Alte Drucke Bibliothek ...,Universität Basel - BS,52627.00
7,Schweiz. Thesauruskommission,"NPO (Biblioth., Museen, Verwalt.) - NPO",120042.00


### 2.2 Data Filtering 
Some cells in the column **University** contains the value ** Nicht zuteilbar - NA **. According to Google Translate, "Nicht zuteilbar" means "Not Assignable", which we interpret as no university could be assigned to that data. Thus we have no choice but to filter out all cells contain *Nicht zuteilbar - NA* in their **University** field.

We also have a similiar problem with the value ** NPO (Biblioth., Museen, Verwalt.) - NPO ** in the **University** column, we will not be able to extract a canton location from this.

Therefore, we set both of these cells to 'None' for further cleaning. 

In [99]:
data_of_interest.replace(['Nicht zuteilbar - NA', 'NPO (Biblioth., Museen, Verwalt.) - NPO'], [None, None], inplace=True)
data_of_interest.head()

,Institution,University,Approved Amount
"﻿""Project Number""",,,
1,NaN,None,11619.00
4,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,41022.00
5,Kommission für das Corpus philosophorum medii ...,None,79732.00
6,Abt. Handschriften und Alte Drucke Bibliothek ...,Universität Basel - BS,52627.00
7,Schweiz. Thesauruskommission,None,120042.00


We can see that some row have neither *Institution* or *University* information.

In [100]:
data_of_interest.loc[20008]

Institution                            NaN
University                             NaN
Approved Amount    data not included in P3
Name: 20008, dtype: object

These row are therefore useless for us, so we can remove them.

In [101]:
# We can use the 'thresh=2' because we know that the column 'Approved Amount' doesn't contain null values.
# Drop the row if it contains 2 na(NaN or None) values in both column University and Institution
data_of_interest = data_of_interest.dropna(axis=0, how="all", thresh=2)
data_of_interest.shape

(59982, 3)

Note that, as seen above, the column *Approved Amount* has no null values. But it doesn't only contains number...

In [102]:
data_of_interest['Approved Amount'].describe()

count                       59982
unique                      35492
top       data not included in P3
freq                        10244
Name: Approved Amount, dtype: object

As we can see, some rows contains a string *"data not included in P3"*. We have no choice but to delete rows which contain this value in the **Approved Amount** column. We only keep rows where the *Approved Amount* value can be cast to numeric.

In [103]:
# The "errors=coerce" will replace all values that cannot be cast to numeric by a null value.
# So we simply need to get ride of all these null values
missing_amount = pd.to_numeric(data_of_interest['Approved Amount'], errors="coerce").isnull()
data_clean = data_of_interest[~ missing_amount]

print("Type of 'Approved Amount column:", data_clean['Approved Amount'].dtypes)
print("Any null values? ",data_clean['Approved Amount'].isnull().any())
data_clean.head()

Type of 'Approved Amount column: object
Any null values?  False


,Institution,University,Approved Amount
"﻿""Project Number""",,,
4,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,41022.00
5,Kommission für das Corpus philosophorum medii ...,None,79732.00
6,Abt. Handschriften und Alte Drucke Bibliothek ...,Universität Basel - BS,52627.00
7,Schweiz. Thesauruskommission,None,120042.00
8,"Séminaire de politique économique, d'économie ...",Université de Fribourg - FR,53009.00


In [104]:
data_clean.shape

(49738, 3)

## 3. Add "Canton" information 

Based on the **Institution** and **University** information, we will try to find the corresponding **Canton** information for each row.

In [105]:
# The list of Universities with the corresponding number of entries
a = data_clean.groupby('University').size()
print("Total:", a.sum())
a.sort_values(ascending=False)

Total: 46810


University
Universität Zürich - ZH                               6754
Université de Genève - GE                             6379
ETH Zürich - ETHZ                                     6143
Universität Bern - BE                                 5460
Universität Basel - BS                                4737
EPF Lausanne - EPFL                                   4406
Université de Lausanne - LA                           4090
Université de Fribourg - FR                           2078
Université de Neuchâtel - NE                          1589
Paul Scherrer Institut - PSI                           537
Firmen/Privatwirtschaft - FP                           492
Universität St. Gallen - SG                            426
Università della Svizzera italiana - USI               346
Eidg. Anstalt für Wasserversorgung - EAWAG             333
HES de Suisse occidentale - HES-SO                     271
Zürcher Fachhochschule (ohne PH) - ZFH                 260
Eidg. Material und Prüfungsanstalt - EMPA    

### 3.1 Retrieve Canton based on University Name
First, we will use the **University** field to try to retrieve it's **Canton** information.
We start by creating a Dataframe with all Universitiy names as the index.

In [106]:
df = pd.DataFrame()
df['University'] = data_clean.University.unique()
df.index = df['University']
df.head()

,University
University,
Université de Genève - GE,Université de Genève - GE
NaN,None
Universität Basel - BS,Universität Basel - BS
Université de Fribourg - FR,Université de Fribourg - FR
Universität Zürich - ZH,Universität Zürich - ZH


Some of these university value are compose of two parts: ***University Name*** and ***University "Code"***.
Therefore we split each of these values. If no split possible, set *University Code* to *None*

In [107]:
institution = "**Institution**"
no_value = "**No_Value"

df['University Name'] = None
df['University Code'] = None

for index, row in df.iterrows():
    
    x = row.University
    if x == x and x: # Assert x is not Nan nor None
        try:
            a,b = x.split('-')
        except:
            a,b = x,no_value
    else:
        #x is NaN, but the 'Instition' value is not!
        a = b = institution
    
    row['University Name'] = a
    row['University Code'] = b
        
df.head()

,University,University Name,University Code
University,,,
Université de Genève - GE,Université de Genève - GE,Université de Genève,GE
NaN,None,**Institution**,**Institution**
Universität Basel - BS,Universität Basel - BS,Universität Basel,BS
Université de Fribourg - FR,Université de Fribourg - FR,Université de Fribourg,FR
Universität Zürich - ZH,Universität Zürich - ZH,Universität Zürich,ZH


Now we try to use **GeoNames** API to search and retrieve canton information for each University.

In [108]:
geonames_url = 'http://api.geonames.org/search?q='
geonames_url_param = '&country=CH&maxRows=1&username=epfl_interactive_viz&type=json'

def getCanton(name):
    """ Retrieve the Canton information for a given place
    
        @param name: place's name. Must be in Switzerland
        
        @return The canton's string if found or False, otherwise.
    """
    
    url =geonames_url + name + geonames_url_param

    query = parse.quote(url, safe=':/&=?')
    js = json.loads(requests.get(query).text)

    try:
        # Canton information is stored under 'adminCode1' in the JSON given by GeoNames
        return js['geonames'][0]['adminCode1']
    except:
        return False

First we try to find the canton information with the *University Name* information. If *GeoNames* isn't helpful, we try again, but with the ***University Code*** information.

In [109]:
df['Canton'] = None

for index, row in df.iterrows():
    name = row['University Name']
    code = row['University Code']
    
    canton = getCanton(row['University Name'])
    if not canton: # Geonames with the 'University Name' returns false
        canton = getCanton(row['University Code'])
    
    if canton: # If Canton is not null
        row['Canton'] = canton
        
df.head()

,University,University Name,University Code,Canton
University,,,,
Université de Genève - GE,Université de Genève - GE,Université de Genève,GE,GE
NaN,None,**Institution**,**Institution**,None
Universität Basel - BS,Universität Basel - BS,Universität Basel,BS,BS
Université de Fribourg - FR,Université de Fribourg - FR,Université de Fribourg,FR,FR
Universität Zürich - ZH,Universität Zürich - ZH,Universität Zürich,ZH,ZH


In [110]:
df.head(50)

,University,University Name,University Code,Canton
University,,,,
Université de Genève - GE,Université de Genève - GE,Université de Genève,GE,GE
NaN,None,**Institution**,**Institution**,None
Universität Basel - BS,Universität Basel - BS,Universität Basel,BS,BS
Université de Fribourg - FR,Université de Fribourg - FR,Université de Fribourg,FR,FR
Universität Zürich - ZH,Universität Zürich - ZH,Universität Zürich,ZH,ZH
Université de Lausanne - LA,Université de Lausanne - LA,Université de Lausanne,LA,VD
Universität Bern - BE,Universität Bern - BE,Universität Bern,BE,BE
"Eidg. Forschungsanstalt für Wald,Schnee,Land - WSL","Eidg. Forschungsanstalt für Wald,Schnee,Land -...","Eidg. Forschungsanstalt für Wald,Schnee,Land",WSL,ZH
Université de Neuchâtel - NE,Université de Neuchâtel - NE,Université de Neuchâtel,NE,NE


Now that we have the canton information retrieved with *GeoNames* for somes Universities, we can create the column **Canton** in our data.

In [111]:
data_clean['Canton'] = None

for index, row in data_clean.iterrows():
    if row.University == row.University: # Assert that University is not null or NaN
        row['Canton'] = df.loc[row.University, 'Canton']

data_clean.head()

,Institution,University,Approved Amount,Canton
"﻿""Project Number""",,,,
4,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,41022.00,GE
5,Kommission für das Corpus philosophorum medii ...,None,79732.00,None
6,Abt. Handschriften und Alte Drucke Bibliothek ...,Universität Basel - BS,52627.00,BS
7,Schweiz. Thesauruskommission,None,120042.00,None
8,"Séminaire de politique économique, d'économie ...",Université de Fribourg - FR,53009.00,FR


After this first step, we have 88.52% of canton information found.

In [112]:
1 - data_clean.Canton.isnull().sum() / data_clean.shape[0]

0.88517833447263661

In [113]:
nullCanton = data_clean[data_clean.Canton.isnull()]
print(nullCanton.shape)
nullCanton.groupby('University').size().sort_values(ascending=False)
nullCanton.groupby('University').size().sum()

(5711, 4)


2783

In [136]:
df_canton = data_clean

df_canton['Approved Amount'] = df_canton['Approved Amount'].astype('float')

df_canton = df_canton.groupby('Canton').sum()

df_canton['Canton'] = df_canton.index

df_canton.index = range(12)

df_canton

,Approved Amount,Canton
0,1.154283e+08,AG
1,1.519373e+09,BE
2,4.277191e+07,BL
3,1.352251e+09,BS
4,4.575262e+08,FR
5,1.838237e+09,GE
6,4.192589e+07,LU
7,3.832046e+08,NE
8,8.422906e+07,SG
9,2.404008e+07,TI


In [179]:
canton_geo = "Data/ch-cantons.topojson.json"

canton_map = folium.Map(location=[46.8, 8.28], zoom_start=8)

# map.geo_json(geo_path=canton_geo, data=df_canton,
#             columns=['Canton', 'Approved Amount'],
#             key_on='feature.id',
#             fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
#             legend_name='Approved Amount')

canton_map.choropleth(geo_path = canton_geo, data = df_canton,
                             columns = ['Canton', 'Approved Amount'],
                             threshold_scale = [4, 5, 6, 7, 8, 9],
                             key_on = 'feature.id',
                             topojson = 'objects.cantons',
                             fill_color = 'YlGn', fill_opacity = 0.7, line_opacity = 0.2,
                             legend_name = 'Approved Amount')

canton_map.create_map(path='canton.html')

KeyError: 'UR'